# 🎯 Tutorial: Tasks en CrewAI

## Introducción

En el framework CrewAI, una **Task** es una asignación específica completada por un **Agent**. Las tareas proporcionan todos los detalles necesarios para la ejecución, como descripción, agente responsable, herramientas requeridas y más, facilitando una amplia gama de complejidades de acción.

Las tareas dentro de CrewAI pueden ser colaborativas, requiriendo que múltiples agentes trabajen juntos. Esto se maneja a través de las propiedades de la tarea y es orquestado por el proceso del Crew, mejorando el trabajo en equipo y la eficiencia.

### 📋 Contenido del Tutorial

1. [Conceptos Básicos](#conceptos-básicos)
2. [Atributos de las Tareas](#atributos-de-las-tareas)
3. [Creación de Tareas](#creación-de-tareas)
4. [Configuración YAML](#configuración-yaml)
5. [Definición Directa en Código](#definición-directa-en-código)
6. [Salidas de Tareas](#salidas-de-tareas)
7. [Referenciando Otras Tareas](#referenciando-otras-tareas)
8. [Ejecución Asíncrona](#ejecución-asíncrona)
9. [Mecanismo de Callback](#mecanismo-de-callback)
10. [Acceso a Salidas Específicas](#acceso-a-salidas-específicas)
11. [Mecanismo de Override de Herramientas](#mecanismo-de-override-de-herramientas)
12. [Manejo de Errores y Validación](#manejo-de-errores-y-validación)
13. [Creación de Directorios al Guardar Archivos](#creación-de-directorios-al-guardar-archivos)
14. [Ejemplos Prácticos](#ejemplos-prácticos)

---

## Conceptos Básicos

### ¿Qué es una Task?

Una **Task** en CrewAI es una unidad de trabajo que:

- Define una asignación específica para un agente
- Contiene todos los detalles necesarios para la ejecución
- Puede ser colaborativa entre múltiples agentes
- Se puede ejecutar de forma secuencial o jerárquica

### Flujo de Ejecución de Tareas

Las tareas se pueden ejecutar de dos maneras:

1. **Secuencial**: Las tareas se ejecutan en el orden que están definidas
2. **Jerárquico**: Las tareas se asignan a agentes basándose en sus roles y experiencia

```python
crew = Crew(
    agents=[agent1, agent2],
    tasks=[task1, task2],
    process=Process.sequential  # o Process.hierarchical
)
```

## Atributos de las Tareas

| Atributo | Parámetros | Tipo | Descripción |
|----------|------------|------|-------------|
| **Description** | `description` | `str` | Una declaración clara y concisa de lo que implica la tarea |
| **Expected Output** | `expected_output` | `str` | Una descripción detallada de cómo se ve la finalización de la tarea |
| **Name** (opcional) | `name` | `Optional[str]` | Un identificador de nombre para la tarea |
| **Agent** (opcional) | `agent` | `Optional[BaseAgent]` | El agente responsable de ejecutar la tarea |
| **Tools** (opcional) | `tools` | `List[BaseTool]` | Las herramientas/recursos que el agente puede usar para esta tarea |
| **Context** (opcional) | `context` | `Optional[List["Task"]]` | Otras tareas cuyas salidas se usarán como contexto para esta tarea |
| **Async Execution** (opcional) | `async_execution` | `Optional[bool]` | Si la tarea debe ejecutarse de forma asíncrona. Por defecto False |
| **Human Input** (opcional) | `human_input` | `Optional[bool]` | Si la tarea debe tener una revisión humana de la respuesta final del agente. Por defecto False |
| **Markdown** (opcional) | `markdown` | `Optional[bool]` | Si la tarea debe instruir al agente para devolver la respuesta final formateada en Markdown. Por defecto False |
| **Config** (opcional) | `config` | `Optional[Dict[str, Any]]` | Parámetros de configuración específicos de la tarea |
| **Output File** (opcional) | `output_file` | `Optional[str]` | Ruta del archivo para almacenar la salida de la tarea |
| **Create Directory** (opcional) | `create_directory` | `Optional[bool]` | Si se debe crear el directorio para output_file si no existe. Por defecto True |
| **Output JSON** (opcional) | `output_json` | `Optional[Type[BaseModel]]` | Un modelo Pydantic para estructurar la salida JSON |
| **Output Pydantic** (opcional) | `output_pydantic` | `Optional[Type[BaseModel]]` | Un modelo Pydantic para la salida de la tarea |
| **Callback** (opcional) | `callback` | `Optional[Any]` | Función/objeto a ejecutar después de la finalización de la tarea |
| **Guardrail** (opcional) | `guardrail` | `Optional[Callable]` | Función para validar la salida de la tarea antes de proceder a la siguiente tarea |

## Creación de Tareas

Hay dos formas de crear tareas en CrewAI:

1. **Configuración YAML (Recomendado)**: Proporciona una forma más limpia y mantenible de definir tareas
2. **Definición Directa en Código**: Alternativa para casos específicos

### Configuración YAML (Recomendado)

Usar configuración YAML proporciona una forma más limpia y mantenible de definir tareas. Se recomienda encarecidamente usar este enfoque para definir tareas en tus proyectos CrewAI.

Las variables en tus archivos YAML (como `{topic}`) serán reemplazadas con valores de tus entradas cuando ejecutes el crew:

```python
crew.kickoff(inputs={'topic': 'AI Agents'})
```

#### Ejemplo de Configuración YAML

```yaml
# tasks.yaml
research_task:
  description: >
    Conduct a thorough research about {topic}
    Make sure you find any interesting and relevant information given the current year is 2025.
  expected_output: >
    A list with 10 bullet points of the most relevant information about {topic}
  agent: researcher

reporting_task:
  description: >
    Review the context you got and expand each topic into a full section for a report.
    Make sure the report is detailed and contains any and all relevant information.
  expected_output: >
    A fully fledge reports with the mains topics, each with a full section of information.
    Formatted as markdown without '```'
  agent: reporting_analyst
  markdown: true
  output_file: report.md
```

#### Uso en Código Python

Para usar esta configuración YAML en tu código, crea una clase crew que herede de `CrewBase`:

```python
# crew.py
from crewai import Agent, Crew, Process, Task
from crewai.project import CrewBase, agent, crew, task
from crewai_tools import SerperDevTool

@CrewBase
class LatestAiDevelopmentCrew():
    """LatestAiDevelopment crew"""

    @agent
    def researcher(self) -> Agent:
        return Agent(
            config=self.agents_config['researcher'],
            verbose=True,
            tools=[SerperDevTool()]
        )

    @agent
    def reporting_analyst(self) -> Agent:
        return Agent(
            config=self.agents_config['reporting_analyst'],
            verbose=True
        )

    @task
    def research_task(self) -> Task:
        return Task(
            config=self.tasks_config['research_task']
        )

    @task
    def reporting_task(self) -> Task:
        return Task(
            config=self.tasks_config['reporting_task']
        )

    @crew
    def crew(self) -> Crew:
        return Crew(
            agents=[
                self.researcher(),
                self.reporting_analyst()
            ],
            tasks=[
                self.research_task(),
                self.reporting_task()
            ],
            process=Process.sequential
        )
```

**Nota**: Los nombres que uses en tus archivos YAML (`agents.yaml` y `tasks.yaml`) deben coincidir con los nombres de los métodos en tu código Python.

### Definición Directa en Código (Alternativa)

Alternativamente, puedes definir tareas directamente en tu código sin usar configuración YAML:

```python
from crewai import Task

research_task = Task(
    description="""
    Conduct a thorough research about AI Agents.
    Make sure you find any interesting and relevant information given the current year is 2025.
    """,
    expected_output="""
    A list with 10 bullet points of the most relevant information about AI Agents
    """,
    agent=researcher
)

reporting_task = Task(
    description="""
    Review the context you got and expand each topic into a full section for a report.
    Make sure the report is detailed and contains any and all relevant information.
    """,
    expected_output="""
    A fully fledge reports with the mains topics, each with a full section of information.
    """,
    agent=reporting_analyst,
    markdown=True,  # Habilitar formato markdown para la salida final
    output_file="report.md"
)
```

Puedes asignar directamente un `agent` o dejar que el proceso `hierarchical` de CrewAI decida basándose en roles, disponibilidad, etc.

## Salidas de Tareas

Entender las salidas de las tareas es crucial para construir flujos de trabajo de IA efectivos. CrewAI proporciona una forma estructurada de manejar los resultados de las tareas a través de la clase `TaskOutput`, que admite múltiples formatos de salida y puede pasarse fácilmente entre tareas.

La salida de una tarea en el framework CrewAI está encapsulada dentro de la clase `TaskOutput`. Esta clase proporciona una forma estructurada de acceder a los resultados de una tarea, incluyendo varios formatos como salida raw, JSON y modelos Pydantic.

Por defecto, `TaskOutput` solo incluirá la salida raw, pero puedes configurar salidas adicionales como JSON o Pydantic según tus necesidades.

## Referenciando Otras Tareas

En CrewAI, la salida de una tarea se transmite automáticamente a la siguiente, pero puedes definir específicamente qué salidas de tareas, incluyendo múltiples, deben usarse como contexto para otra tarea. Esto es útil cuando tienes una tarea que depende de la salida de otra tarea que no se realiza inmediatamente después.

Esto se hace a través del atributo `context` de la tarea:

```python
research_ai_task = Task(
    description="Research the latest developments in AI",
    expected_output="A list of recent AI developments",
    async_execution=True,
    agent=research_agent,
    tools=[search_tool]
)

research_ops_task = Task(
    description="Research the latest developments in AI Ops",
    expected_output="A list of recent AI Ops developments",
    async_execution=True,
    agent=research_agent,
    tools=[search_tool]
)

write_blog_task = Task(
    description="Write a full blog post about the importance of AI and its latest news",
    expected_output="Full blog post that is 4 paragraphs long",
    agent=writer_agent,
    context=[research_ai_task, research_ops_task]  # Esperará por ambas tareas
)
```

## Ejecución Asíncrona

Puedes definir una tarea para que se ejecute de forma asíncrona. Esto significa que el crew no esperará a que se complete para continuar con la siguiente tarea. Esto es útil para tareas que tardan mucho tiempo en completarse, o que no son cruciales para que se realicen las siguientes tareas.

Puedes usar el atributo `context` para definir en una tarea futura que debe esperar a que se complete la salida de la tarea asíncrona:

```python
list_ideas = Task(
    description="List of 5 interesting ideas to explore for an article about AI.",
    expected_output="Bullet point list of 5 ideas for an article.",
    agent=researcher,
    async_execution=True  # Se ejecutará de forma asíncrona
)

list_important_history = Task(
    description="Research the history of AI and give me the 5 most important events.",
    expected_output="Bullet point list of 5 important events.",
    agent=researcher,
    async_execution=True  # Se ejecutará de forma asíncrona
)

write_article = Task(
    description="Write an article about AI, its history, and interesting ideas.",
    expected_output="A 4 paragraph article about AI.",
    agent=writer,
    context=[list_ideas, list_important_history]  # Esperará por la salida de las dos tareas
)
```

## Mecanismo de Callback

La función callback se ejecuta después de que la tarea se completa, permitiendo que se desencadenen acciones o notificaciones basadas en el resultado de la tarea.

```python
def callback_function(output: TaskOutput):
    # Hacer algo después de que la tarea se complete
    # Ejemplo: Enviar un email al manager
    print(f"""
    Task completed!
    Task: {output.description}
    Output: {output.raw}
    """)

research_task = Task(
    description='Find and summarize the latest AI news',
    expected_output='A bullet list summary of the top 5 most important AI news',
    agent=research_agent,
    tools=[search_tool],
    callback=callback_function
)
```

## Acceso a Salidas Específicas

Una vez que un crew termina de ejecutarse, puedes acceder a la salida de una tarea específica usando el atributo `output` del objeto task:

```python
task1 = Task(
    description='Find and summarize the latest AI news',
    expected_output='A bullet list summary of the top 5 most important AI news',
    agent=research_agent,
    tools=[search_tool]
)

crew = Crew(
    agents=[research_agent],
    tasks=[task1, task2, task3],
    verbose=True
)

result = crew.kickoff()

# Retorna un objeto TaskOutput con la descripción y resultados de la tarea
print(f"""
Task completed!
Task: {task1.output.description}
Output: {task1.output.raw}
""")
```

## Mecanismo de Override de Herramientas

Especificar herramientas en una tarea permite la adaptación dinámica de las capacidades del agente, enfatizando la flexibilidad de CrewAI.

```python
import os
os.environ["SERPER_API_KEY"] = "Your Key"  # serper.dev API key

from crewai import Agent, Task, Crew
from crewai_tools import SerperDevTool

research_agent = Agent(
    role='Researcher',
    goal='Find and summarize the latest AI news',
    backstory="""You're a researcher at a large company. You're responsible for analyzing data and providing insights to the business.""",
    verbose=True
)

# Para realizar una búsqueda semántica para una consulta especificada desde el contenido de un texto a través de internet
search_tool = SerperDevTool()

task = Task(
    description='Find and summarize the latest AI news',
    expected_output='A bullet list summary of the top 5 most important AI news',
    agent=research_agent,
    tools=[search_tool]
)

crew = Crew(
    agents=[research_agent],
    tasks=[task],
    verbose=True
)

result = crew.kickoff()
print(result)
```

Esto demuestra cómo las tareas con herramientas específicas pueden anular el conjunto predeterminado de un agente para una ejecución de tarea personalizada.

## Manejo de Errores y Validación

Mientras se crean y ejecutan tareas, ciertos mecanismos de validación están en su lugar para asegurar la robustez y confiabilidad de los atributos de la tarea. Estos incluyen pero no se limitan a:

- Asegurar que solo un tipo de salida se establezca por tarea para mantener expectativas claras de salida
- Prevenir la asignación manual del atributo `id` para mantener la integridad del sistema de identificadores únicos

Estas validaciones ayudan a mantener la consistencia y confiabilidad de las ejecuciones de tareas dentro del framework crewAI.

## Creación de Directorios al Guardar Archivos

El parámetro `create_directory` controla si CrewAI debe crear automáticamente directorios al guardar salidas de tareas en archivos. Esta característica es particularmente útil para organizar salidas y asegurar que las rutas de archivos estén correctamente estructuradas, especialmente cuando se trabaja con jerarquías de proyectos complejas.

### Comportamiento por Defecto

Por defecto, `create_directory=True`, lo que significa que CrewAI creará automáticamente cualquier directorio faltante en la ruta del archivo de salida:

```python
# Comportamiento por defecto - los directorios se crean automáticamente
report_task = Task(
    description='Generate a comprehensive market analysis report',
    expected_output='A detailed market analysis with charts and insights',
    agent=analyst_agent,
    output_file='reports/2025/market_analysis.md',  # Crea 'reports/2025/' si no existe
    markdown=True
)
```

### Deshabilitando la Creación de Directorios

Si quieres prevenir la creación automática de directorios y asegurar que el directorio ya existe, establece `create_directory=False`:

```python
# Modo estricto - el directorio debe existir ya
strict_output_task = Task(
    description='Save critical data that requires existing infrastructure',
    expected_output='Data saved to pre-configured location',
    agent=data_agent,
    output_file='secure/vault/critical_data.json',
    create_directory=False  # Lanzará RuntimeError si 'secure/vault/' no existe
)
```

### Configuración YAML

También puedes configurar este comportamiento en tus definiciones de tareas YAML:

```yaml
# tasks.yaml
analysis_task:
  description: >
    Generate quarterly financial analysis
  expected_output: >
    A comprehensive financial report with quarterly insights
  agent: financial_analyst
  output_file: reports/quarterly/q4_2024_analysis.pdf
  create_directory: true  # Crear automáticamente el directorio 'reports/quarterly/'

audit_task:
  description: >
    Perform compliance audit and save to existing audit directory
  expected_output: >
    A compliance audit report
  agent: auditor
  output_file: audit/compliance_report.md
  create_directory: false  # El directorio debe existir ya
```

### Casos de Uso

**Creación Automática de Directorios (`create_directory=True`):**
- Entornos de desarrollo y prototipado
- Generación dinámica de reportes con carpetas basadas en fecha
- Flujos de trabajo automatizados donde la estructura de directorios puede variar
- Aplicaciones multi-tenant con carpetas específicas de usuario

**Gestión Manual de Directorios (`create_directory=False`):**
- Entornos de producción con controles estrictos del sistema de archivos
- Aplicaciones sensibles a la seguridad donde los directorios deben estar pre-configurados
- Sistemas con requisitos específicos de permisos
- Entornos de cumplimiento donde la creación de directorios está auditada

### Manejo de Errores

Cuando `create_directory=False` y el directorio no existe, CrewAI lanzará un `RuntimeError`:

```python
try:
    result = crew.kickoff()
except RuntimeError as e:
    # Manejar error de directorio faltante
    print(f"Directory creation failed: {e}")
    # Crear directorio manualmente o usar ubicación alternativa
```

## Ejemplos Prácticos

### Ejemplo 1: Tarea Simple de Investigación

```python
from crewai import Agent, Task, Crew
from crewai_tools import SerperDevTool

# Configurar API key
import os
os.environ["SERPER_API_KEY"] = "your_api_key_here"

# Crear agente investigador
researcher = Agent(
    role='AI Research Analyst',
    goal='Find and analyze the latest developments in artificial intelligence',
    backstory="""You are an expert research analyst specializing in AI technologies. 
    You have a deep understanding of machine learning, neural networks, and emerging AI trends.""",
    verbose=True,
    tools=[SerperDevTool()]
)

# Crear tarea de investigación
research_task = Task(
    description="""
    Research the latest developments in artificial intelligence for 2025.
    Focus on:
    - New AI models and breakthroughs
    - Industry applications and use cases
    - Market trends and investments
    - Ethical considerations and regulations
    """,
    expected_output="""
    A comprehensive report with:
    - 5 key AI developments with detailed explanations
    - 3 major industry applications
    - Market analysis summary
    - Ethical considerations overview
    Format as bullet points with clear sections.
    """,
    agent=researcher,
    markdown=True,
    output_file="ai_research_report.md"
)

# Crear crew y ejecutar
crew = Crew(
    agents=[researcher],
    tasks=[research_task],
    verbose=True
)

result = crew.kickoff()
print(result)
```

### Ejemplo 2: Tareas Secuenciales con Múltiples Agentes

```python
from crewai import Agent, Task, Crew, Process
from crewai_tools import SerperDevTool

# Agente investigador
researcher = Agent(
    role='Market Researcher',
    goal='Conduct thorough market research and gather data',
    backstory='Expert in market analysis with 10+ years of experience',
    verbose=True,
    tools=[SerperDevTool()]
)

# Agente analista
analyst = Agent(
    role='Data Analyst',
    goal='Analyze data and create insights',
    backstory='Skilled data analyst with expertise in statistical analysis',
    verbose=True
)

# Agente escritor
writer = Agent(
    role='Content Writer',
    goal='Create compelling content based on research and analysis',
    backstory='Experienced content writer with expertise in technical writing',
    verbose=True
)

# Tarea 1: Investigación
research_task = Task(
    description="""
    Research the current state of renewable energy markets worldwide.
    Focus on solar, wind, and battery storage technologies.
    Gather data on market size, growth rates, key players, and future projections.
    """,
    expected_output="""
    A detailed research report containing:
    - Market size and growth data for each technology
    - Key players and market leaders
    - Regional analysis
    - Future projections and trends
    """,
    agent=researcher
)

# Tarea 2: Análisis
analysis_task = Task(
    description="""
    Analyze the research data and identify key insights and opportunities.
    Create market analysis with SWOT analysis and recommendations.
    """,
    expected_output="""
    A comprehensive analysis including:
    - Key market insights
    - SWOT analysis
    - Investment opportunities
    - Risk assessment
    - Strategic recommendations
    """,
    agent=analyst,
    context=[research_task]
)

# Tarea 3: Escritura
writing_task = Task(
    description="""
    Create a comprehensive market report based on the research and analysis.
    Make it engaging and professional for executive audience.
    """,
    expected_output="""
    A professional market report with:
    - Executive summary
    - Detailed market analysis
    - Visual recommendations
    - Actionable insights
    Format as a professional business document.
    """,
    agent=writer,
    context=[analysis_task],
    markdown=True,
    output_file="renewable_energy_market_report.md"
)

# Crear crew y ejecutar
crew = Crew(
    agents=[researcher, analyst, writer],
    tasks=[research_task, analysis_task, writing_task],
    process=Process.sequential,
    verbose=True
)

result = crew.kickoff()
print(result)
```

### Ejemplo 3: Tareas Asíncronas

```python
from crewai import Agent, Task, Crew
from crewai_tools import SerperDevTool

# Agente investigador
researcher = Agent(
    role='Research Specialist',
    goal='Conduct parallel research on multiple topics',
    backstory='Expert researcher capable of handling multiple research streams',
    verbose=True,
    tools=[SerperDevTool()]
)

# Agente sintetizador
synthesizer = Agent(
    role='Content Synthesizer',
    goal='Combine and synthesize multiple research outputs',
    backstory='Expert in combining information from multiple sources',
    verbose=True
)

# Tarea asíncrona 1: Investigar IA
ai_research = Task(
    description="Research the latest developments in artificial intelligence",
    expected_output="Comprehensive list of AI developments with explanations",
    agent=researcher,
    async_execution=True
)

# Tarea asíncrona 2: Investigar blockchain
blockchain_research = Task(
    description="Research the latest developments in blockchain technology",
    expected_output="Comprehensive list of blockchain developments with explanations",
    agent=researcher,
    async_execution=True
)

# Tarea asíncrona 3: Investigar IoT
iot_research = Task(
    description="Research the latest developments in Internet of Things",
    expected_output="Comprehensive list of IoT developments with explanations",
    agent=researcher,
    async_execution=True
)

# Tarea de síntesis (espera por todas las investigaciones)
synthesis_task = Task(
    description="""
    Combine all research findings and create a comprehensive technology trends report.
    Identify connections between AI, blockchain, and IoT developments.
    """,
    expected_output="""
    A comprehensive technology trends report covering:
    - AI developments and their impact
    - Blockchain innovations and applications
    - IoT advancements and use cases
    - Cross-technology synergies and opportunities
    - Future predictions and recommendations
    """,
    agent=synthesizer,
    context=[ai_research, blockchain_research, iot_research],
    markdown=True,
    output_file="technology_trends_report.md"
)

# Crear crew y ejecutar
crew = Crew(
    agents=[researcher, synthesizer],
    tasks=[ai_research, blockchain_research, iot_research, synthesis_task],
    verbose=True
)

result = crew.kickoff()
print(result)
```

## Resumen

En este tutorial hemos cubierto todos los aspectos importantes de las **Tasks** en CrewAI:

### ✅ Conceptos Clave Aprendidos:

1. **Definición**: Las tareas son unidades de trabajo específicas asignadas a agentes
2. **Atributos**: Descripción, salida esperada, agente, herramientas, contexto, etc.
3. **Creación**: Dos métodos principales (YAML recomendado y código directo)
4. **Flujos**: Secuencial y jerárquico
5. **Características Avanzadas**:
   - Ejecución asíncrona
   - Referenciando otras tareas
   - Callbacks
   - Override de herramientas
   - Manejo de archivos y directorios

### 🚀 Próximos Pasos:

1. **Practica**: Crea tus propias tareas usando los ejemplos proporcionados
2. **Experimenta**: Prueba diferentes configuraciones y flujos
3. **Integra**: Combina tareas con agentes y crews para crear flujos de trabajo complejos
4. **Optimiza**: Usa callbacks y validaciones para mejorar la robustez

### 📚 Recursos Adicionales:

- [Documentación oficial de CrewAI](https://docs.crewai.com/)
- [Guía de Agentes](https://docs.crewai.com/en/concepts/agents.md)
- [Guía de Crews](https://docs.crewai.com/en/concepts/crews.md)
- [Herramientas disponibles](https://docs.crewai.com/en/tools/)

---

**¡Felicidades!** 🎉 Has completado el tutorial sobre Tasks en CrewAI. Ahora tienes las herramientas necesarias para crear tareas efectivas y construir flujos de trabajo de IA robustos.

**Recuerda**: La práctica es la mejor manera de dominar estos conceptos. ¡Empieza con ejemplos simples y ve construyendo hacia casos de uso más complejos!